In [1]:
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical

In [2]:
# Load FER2013 dataset
data = pd.read_csv("fer2013.csv")

In [3]:
# Preprocess data
X = []
y = []
for index, row in data.iterrows():
    pixels = np.array(row['pixels'].split(' '), dtype='uint8')
    image = pixels.reshape((48, 48))
    X.append(image)
    y.append(row['emotion'])

In [4]:
X = np.array(X)
y = np.array(y)

In [5]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Reshape data and normalize
X_train = X_train.reshape(-1, 48, 48, 1) / 255.0
X_test = X_test.reshape(-1, 48, 48, 1) / 255.0

In [7]:
# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [8]:
# Define CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [9]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/20
449/449 [==============================] - 43s 20ms/step - loss: 1.7374 - accuracy: 0.2959 - val_loss: 1.5372 - val_accuracy: 0.4007
Epoch 2/20
449/449 [==============================] - 7s 15ms/step - loss: 1.4784 - accuracy: 0.4304 - val_loss: 1.3668 - val_accuracy: 0.4830
Epoch 3/20
449/449 [==============================] - 7s 15ms/step - loss: 1.3558 - accuracy: 0.4848 - val_loss: 1.2834 - val_accuracy: 0.5125
Epoch 4/20
449/449 [==============================] - 7s 15ms/step - loss: 1.2894 - accuracy: 0.5117 - val_loss: 1.2666 - val_accuracy: 0.5202
Epoch 5/20
449/449 [==============================] - 7s 15ms/step - loss: 1.2313 - accuracy: 0.5338 - val_loss: 1.2085 - val_accuracy: 0.5428
Epoch 6/20
449/449 [==============================] - 7s 14ms/step - loss: 1.1746 - accuracy: 0.5513 - val_loss: 1.1739 - val_accuracy: 0.5561
Epoch 7/20
449/449 [==============================] - 6s 14ms/step - loss: 1.1327 - accuracy: 0.5747 - val_loss: 1.1745 - val_accuracy: 0.555

In [11]:
# Save the model
model.save("emotion_detection_model.h5")

In [1]:
import tensorflow as tf

In [2]:
# Load the trained Keras model
model = tf.keras.models.load_model("emotion_detection_model.h5")

# Convert the Keras model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open("emotion_detection_model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\JIANNE~1\AppData\Local\Temp\tmpw2rnlhgf\assets


INFO:tensorflow:Assets written to: C:\Users\JIANNE~1\AppData\Local\Temp\tmpw2rnlhgf\assets
